In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
import sys
import random
from sklearn.preprocessing import StandardScaler
from IPython.display import display, Math
from torch.utils.data import DataLoader, Dataset
import time
import json

from utils import *
from model_utils import *

sys_epsilon = sys.float_info.epsilon

# Data Preprocessing

In this section, we preprocess the CFD data collected from DSM-based LES of flow past a sphere at $\mathcal{R}e=10^3$, $5\times 10^3$, and $10^4$. The data for each Reynolds number is collected from $t=300$ to $320$. From $t=300$ to $310$, data is collected at every ten variable time-steps. However, from $t=310$ to $320$, data is collected with $\Delta t =1$.

In [2]:
paths = ["/home/hmarefat/scratch/torchFOAM/Case_dS/postProcessing/fieldData.dat",
         "/home/hmarefat/scratch/torchFOAM/Case_dS_R53/postProcessing/fieldData.dat",
         "/home/hmarefat/scratch/torchFOAM/Case_dS_R4/postProcessing/fieldData.dat"]
nCols = 26
names = ['_R3', '_R53', '_R4']

In [ ]:
random.seed(35)

count = 1000000
Indx_R3 = returnRandmIndx(4956526, count)
Indx_R53 = returnRandmIndx(5900626, count)
Indx_R4 = returnRandmIndx(4012425, count)

In [ ]:
Indx_R3[0:10]

In [ ]:
random.shuffle(Indx_R3)

In [ ]:
Indx_R3[0:10]

In [ ]:
index_data = []

for indx in [Indx_R3, Indx_R53, Indx_R4]:
    seen, unseen = splitterIndx(indx)
    index_data.append((seen, unseen))

index_array = np.array(index_data, dtype=object)

In [ ]:
index_array[2,0].shape, index_array[2,1].shape

In [ ]:
data = []

for i, path in enumerate(paths):
    name = path.split('/')[-1][:-4]+names[i]
    print(f'Starting to preprocess dataset {name}')
    data.clear()
    
    with open(path, "r") as f:
        next(f)
        for line in f:
            l = line.split()
            try:
                data.append([float(x) for x in l]) #(l)
            except ValueError as e:
                print(f"Error converting line to float: {line.strip()} - {e}")
                continue
                
    #data.pop(0)
    print('Reading raw file is done!')
    ds = np.array(data) 
    
    ds_seen = ds[index_array[i,0]]
    ds_unseen = ds[index_array[i,1]]
    
    scaler(name+'_seen', ds_seen)
    scaler(name+'_unseen', ds_unseen)
    
    print('\n')

# Data Preparation

In [2]:
headers = ["t",                                             # time
           "X", "Y", "Z",                                   # spacial coordinates
           "Ux", "Uy", "Uz",                                # velocity components
           "G1", "G2", "G3", "G4", "G5", "G6",              # velocity gradient tensor components
           "S1", "S2", "S3", "S4", "S5", "S6",              # strain rate tensor compnents
           "UUp1", "UUp2", "UUp3", "UUp4", "UUp5", "UUp6",  # resolved Reynolds stress tensor components
           "Cs"]                                            # Smagorinsky coefficient

### Dataset Loading

#### $\mathcal Re = 10^3$

In [3]:
# Re = 10^3 seen
dSn_R103_seen_means = pd.read_csv('../processedDatasets/fieldData_R3_seen_means.txt', sep=' ', names=headers) 
dSn_R103_seen_scales = pd.read_csv('../processedDatasets/fieldData_R3_seen_scales.txt', sep=' ', names=headers) 
dSn_R103_seen = pd.read_csv('../processedDatasets/fieldData_R3_seen_norm.txt', sep=' ', names=headers)
dS_R103_seen = pd.read_csv('../processedDatasets/fieldData_R3_seen.txt', sep=' ', names=headers)

# Re = 10^3 unseen
dSn_R103_unseen_means = pd.read_csv('../processedDatasets/fieldData_R3_unseen_means.txt', sep=' ', names=headers) 
dSn_R103_unseen_scales = pd.read_csv('../processedDatasets/fieldData_R3_unseen_scales.txt', sep=' ', names=headers) 
dSn_R103_unseen = pd.read_csv('../processedDatasets/fieldData_R3_unseen_norm.txt', sep=' ', names=headers)
dS_R103_unseen = pd.read_csv('../processedDatasets/fieldData_R3_unseen.txt', sep=' ', names=headers)

#### $\mathcal Re = 5\times 10^3$

In [4]:
# Re = 5 x 10^3 seen
dSn_R503_seen_means = pd.read_csv('../processedDatasets/fieldData_R53_seen_means.txt', sep=' ', names=headers) 
dSn_R503_seen_scales = pd.read_csv('../processedDatasets/fieldData_R53_seen_scales.txt', sep=' ', names=headers) 
dSn_R503_seen = pd.read_csv('../processedDatasets/fieldData_R53_seen_norm.txt', sep=' ', names=headers)
dS_R503_seen = pd.read_csv('../processedDatasets/fieldData_R53_seen.txt', sep=' ', names=headers)

# Re = 5 x 10^3 unseen
dSn_R503_unseen_means = pd.read_csv('../processedDatasets/fieldData_R53_unseen_means.txt', sep=' ', names=headers) 
dSn_R503_unseen_scales = pd.read_csv('../processedDatasets/fieldData_R53_unseen_scales.txt', sep=' ', names=headers) 
dSn_R503_unseen = pd.read_csv('../processedDatasets/fieldData_R53_unseen_norm.txt', sep=' ', names=headers)
dS_R503_unseen = pd.read_csv('../processedDatasets/fieldData_R53_unseen.txt', sep=' ', names=headers)

#### $\mathcal Re = 10^4$

In [5]:
# Re = 10^4 seen
dSn_R104_seen_means = pd.read_csv('../processedDatasets/fieldData_R4_seen_means.txt', sep=' ', names=headers) 
dSn_R104_seen_scales = pd.read_csv('../processedDatasets/fieldData_R4_seen_scales.txt', sep=' ', names=headers) 
dSn_R104_seen = pd.read_csv('../processedDatasets/fieldData_R4_seen_norm.txt', sep=' ', names=headers)
dS_R104_seen = pd.read_csv('../processedDatasets/fieldData_R4_seen.txt', sep=' ', names=headers)

# Re = 10^4 unseen
dSn_R104_unseen_means = pd.read_csv('../processedDatasets/fieldData_R4_unseen_means.txt', sep=' ', names=headers) 
dSn_R104_unseen_scales = pd.read_csv('../processedDatasets/fieldData_R4_unseen_scales.txt', sep=' ', names=headers) 
dSn_R104_unseen = pd.read_csv('../processedDatasets/fieldData_R4_unseen_norm.txt', sep=' ', names=headers)
dS_R104_unseen = pd.read_csv('../processedDatasets/fieldData_R4_unseen.txt', sep=' ', names=headers)

# Model Configuration Setup

In this section, we consider the different model configurations as presented in the following table.

$$
\begin{array}{|l|c|c|c|c|}
    \hline
    \textbf{Model} & \textbf{Inputs} & \textbf{No. of Inputs} & \textbf{Outputs} & \textbf{No. of Outputs} \\
    \hline
    \mathbf{M1} & u_i \, \text{and} \, \mathcal{S}_{ij} & 9 & c_s & 1\\
    \mathbf{M2} & \mathcal{G}_{ij} \, \text{and} \, \mathcal{S}_{ij} & 12 & c_s & 1 \\
    \mathbf{M3} & u_i \, \text{and} \, \tau^{'}_{ij} & 9 & c_s & 1 \\
    \mathbf{M4} & \mathcal{G}_{ij} \, \text{and} \, \tau^{'}_{ij} & 12 & c_s & 1 \\
    \hline
\end{array}
$$

In [6]:
R103 = dSn_R103_seen
R503 = dSn_R503_seen
R104 = dSn_R104_seen

R103_un = dSn_R103_unseen
R503_un = dSn_R503_unseen
R104_un = dSn_R104_unseen

In [7]:
M1_headers = ['Ux', 'Uy', 'Uz', 'S1',  'S2', 'S3', 'S4', 'S5', 'S6', 'Cs']
M2_headers = ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'S1',  'S2', 'S3', 'S4', 'S5', 'S6', 'Cs']
M3_headers = ['Ux', 'Uy', 'Uz', 'UUp1',  'UUp2', 'UUp3', 'UUp4', 'UUp5', 'UUp6', 'Cs']
M4_headers = ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'UUp1',  'UUp2', 'UUp3', 'UUp4', 'UUp5', 'UUp6', 'Cs']

M1_103 = R103.filter(M1_headers, axis=1)
M2_103 = R103.filter(M2_headers, axis=1)
M3_103 = R103.filter(M3_headers, axis=1)
M4_103 = R103.filter(M4_headers, axis=1)

M1_503 = R503.filter(M1_headers, axis=1)
M2_503 = R503.filter(M2_headers, axis=1)
M3_503 = R503.filter(M3_headers, axis=1)
M4_503 = R503.filter(M4_headers, axis=1)

M1_104 = R104.filter(M1_headers, axis=1)
M2_104 = R104.filter(M2_headers, axis=1)
M3_104 = R104.filter(M3_headers, axis=1)
M4_104 = R104.filter(M4_headers, axis=1)

M1_103_test = R103_un.filter(M1_headers, axis=1)
M2_103_test = R103_un.filter(M2_headers, axis=1)
M3_103_test = R103_un.filter(M3_headers, axis=1)
M4_103_test = R103_un.filter(M4_headers, axis=1)

M1_503_test = R503_un.filter(M1_headers, axis=1)
M2_503_test = R503_un.filter(M2_headers, axis=1)
M3_503_test = R503_un.filter(M3_headers, axis=1)
M4_503_test = R503_un.filter(M4_headers, axis=1)

M1_104_test = R104_un.filter(M1_headers, axis=1)
M2_104_test = R104_un.filter(M2_headers, axis=1)
M3_104_test = R104_un.filter(M3_headers, axis=1)
M4_104_test = R104_un.filter(M4_headers, axis=1)

# Model Training

In [24]:
dt = M2_104
dt_name = namestr(M2_104, globals())[0]

In [25]:
output_size = 1
input_size = dt.shape[1] - output_size 
neurons_per_layer = [60, 60, 60, 60, 60] 
hidden_layers = len(neurons_per_layer) 

split_sz = 0.8
mask = np.random.rand(len(dt)) < split_sz
train = dt[mask].reset_index(drop=True) 
val = dt[~mask].reset_index(drop=True)

In [26]:
train

,G1,G2,G3,G4,G5,G6,S1,S2,S3,S4,S5,S6,Cs
0,-0.229502,-0.082355,-0.049943,0.005463,0.058222,-0.131098,-0.229502,-0.025800,0.028246,0.058222,0.015112,0.318217,-0.289756
1,-0.618960,-0.582423,-0.004830,0.100815,-0.213630,-0.717968,-0.618960,-0.119099,0.293541,-0.213630,-1.118656,-0.994835,0.266733
2,-0.030994,0.652822,-0.045647,-0.148220,0.266144,-0.168707,-0.030994,0.097606,-0.309391,0.266144,-0.407072,-0.144934,-1.158358
3,-0.181946,-0.020944,-0.226529,-0.082291,0.444896,0.366172,-0.181946,-0.091832,-0.215805,0.444896,0.334084,0.031496,-0.686264
4,-0.671802,-0.761704,0.023589,0.006468,-0.524679,-0.105963,-0.671802,-0.283521,0.118334,-0.524679,-0.116864,-0.115713,0.131864
...,...,...,...,...,...,...,...,...,...,...,...,...,...
560431,0.209342,0.864437,-1.144578,0.389387,-0.105700,-0.077340,0.209342,0.726027,-0.378531,-0.105700,-0.226626,-1.097109,0.396064
560432,0.210802,0.010281,1.138833,0.914302,0.851010,-1.320538,0.210802,0.935590,1.063056,0.851010,-0.910689,-0.672601,0.372382
560433,0.719513,-0.389316,-0.419565,-0.152395,-0.146498,-0.100599,0.719513,-0.303570,0.596391,-0.146498,-0.086341,0.096018,-0.127999
560434,0.183035,0.044767,0.739180,0.015846,-0.085614,-0.129985,0.183035,0.033198,0.333166,-0.085614,-0.219804,0.220753,-0.469827


In [27]:
val

,G1,G2,G3,G4,G5,G6,S1,S2,S3,S4,S5,S6,Cs
0,-0.030288,-0.099401,-0.093050,-0.044349,0.073241,0.071077,-0.030288,-0.083051,-0.075719,0.073241,0.106465,0.140135,-1.004127
1,-0.197695,-0.242680,0.244637,0.835817,-1.329357,0.323531,-0.197695,0.759268,0.355617,-1.329357,0.850910,0.474040,0.517645
2,-0.838453,0.744977,-0.456138,0.865464,0.259171,-0.538470,-0.838453,1.165651,0.559152,0.259171,-0.830657,0.649751,0.893781
3,0.178851,-0.154611,0.137452,-0.041943,-0.064516,0.120205,0.178851,-0.101627,0.091542,-0.064516,0.205727,0.084008,0.066254
4,1.400678,0.152804,0.373646,-1.417872,-1.430720,3.650052,1.400678,-1.386618,-0.588300,-1.430720,1.476487,0.351265,0.767310
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139559,0.006266,0.756778,1.014310,-0.518608,-0.262392,-0.849416,0.006266,-0.240226,-0.395148,-0.262392,-0.815884,0.223182,0.469328
139560,-0.325784,-0.157061,-0.205795,-0.114940,0.322107,0.059542,-0.325784,-0.176944,0.070921,0.322107,0.020693,0.203430,0.006298
139561,-0.340133,1.330936,-4.137219,0.948081,-0.129908,3.387646,-0.340133,1.473013,-1.066588,-0.129908,2.276898,2.292677,0.281431
139562,-0.420393,-1.029066,1.392877,-2.040311,-0.480857,0.129104,-0.420393,-2.471025,0.995343,-0.480857,-0.797385,0.548235,0.205607


In [28]:
batch_sz_trn = 4096
batch_sz_val = int(batch_sz_trn / 4)

train_dataset = MyDataset(train)
val_dataset = MyDataset(val)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_sz_trn, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=batch_sz_val, shuffle=True)

In [29]:
data_iter = iter(train_loader)
next(data_iter)[0]

tensor([-9.8697,  5.2901,  3.1248, -6.9723,  5.0522,  3.3308, -9.8697, -5.0900,
        -3.2410,  5.0522,  7.3159,  2.7096, -0.6950], dtype=torch.float64)

In [30]:
model = MLPModel(input_size=input_size, 
                 output_size=output_size, 
                 hidden_layers=hidden_layers, 
                 neurons_per_layer=neurons_per_layer)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=3, gamma=0.2)

device = torch.device("cuda")
model.to(device)
model.double()

MLPModel(
  (block): Sequential(
    (0): Linear(in_features=12, out_features=60, bias=True)
    (1): ReLU()
    (2): Linear(in_features=60, out_features=60, bias=True)
    (3): ReLU()
    (4): Linear(in_features=60, out_features=60, bias=True)
    (5): ReLU()
    (6): Linear(in_features=60, out_features=60, bias=True)
    (7): ReLU()
    (8): Linear(in_features=60, out_features=60, bias=True)
    (9): ReLU()
    (10): Linear(in_features=60, out_features=1, bias=True)
  )
)

In [ ]:
epochs = 6000
best_loss = float('inf')
PATH = f"./best_model_{dt_name}.pt"

early_stopper = EarlyStopper(patience=300, path=PATH)
history = {
    "train_loss": [],
    "val_loss": [],
    "train_coefficient": [],
    "val_coefficient": [],
    "learning_rates": [],
    "epoch_times": []
}


for epoch in range(epochs):
    start_time = time.time()
    model.train()
    Loss_train = 0 
    coeff_train = 0

    with tqdm(train_loader, unit="batch") as trainer:
        for batch in trainer:
            trainer.set_description("Train")
            train_feat = batch[:, 0:-1].to(device)
            train_labs = batch[:, -1].to(device)
            train_pred = model(train_feat).squeeze()
            train_loss = torch.nn.functional.mse_loss(train_pred, train_labs)
            train_coef = coeff_determination(train_pred, train_labs) 

            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

            Loss_train += train_loss.item()
            coeff_train += train_coef.item()

        Loss_train /= len(trainer)
        coeff_train /= len(trainer)

    model.eval()
    Loss_val = 0
    coeff_val = 0
    with tqdm(val_loader, unit="batch") as validator:
        for batch in validator:
            validator.set_description("Valid")
            val_feat = batch[:, 0:-1].to(device)
            val_labs = batch[:, -1].to(device)
            val_pred = model(val_feat).squeeze()
            val_loss = torch.nn.functional.mse_loss(val_pred, val_labs)
            val_coef = coeff_determination(val_pred, val_labs) 

            Loss_val += val_loss.item()
            coeff_val += val_coef.item()

        Loss_val /= len(validator)
        coeff_val /= len(validator)
    
    scheduler.step()
    current_lr = optimizer.param_groups[0]['lr']
    history["learning_rates"].append(current_lr)
    
    epoch_duration = time.time() - start_time

    history["train_loss"].append(Loss_train)
    history["val_loss"].append(Loss_val)
    history["train_coefficient"].append(coeff_train)
    history["val_coefficient"].append(coeff_val)
    history["learning_rates"].append(optimizer.param_groups[0]['lr'])
    history["epoch_times"].append(epoch_duration)

    if early_stopper.early_stop(model.state_dict(), Loss_val):             
        print("Early stopping...")
        break
    
    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {Loss_train:.4f}, Train Coeff: {coeff_train:.4f}, "
          f"Val Loss: {Loss_val:.4f}, Val Coeff: {coeff_val:.4f}, Time: {epoch_duration:.2f} sec")

    
with open(f"training_history_{dt_name}.json", "w") as f:
    json.dump(history, f)
    
print(f"Training history saved to 'training_history_{dt_name}.json'")

data_iter = iter(train_loader)
next(data_iter)[:,0:-1]

traced_script_module = torch.jit.trace(model, next(data_iter)[:,0:-1].to(device))
traced_script_module.save(f"traced_model_{dt_name}.pt")

print(f"Training history saved to 'traced_model_{dt_name}.pt'")

Valid: 100%|██████████| 137/137 [00:05<00:00, 26.93batch/s]


Epoch 1/6000, Train Loss: 0.9428, Train Coeff: -1812.9614, Val Loss: 0.9302, Val Coeff: -8.9638, Time: 25.03 sec


Valid: 100%|██████████| 137/137 [00:05<00:00, 26.74batch/s]


Epoch 2/6000, Train Loss: 0.8768, Train Coeff: -7.2783, Val Loss: 0.8949, Val Coeff: -6.2633, Time: 25.08 sec


Valid: 100%|██████████| 137/137 [00:05<00:00, 26.98batch/s]


...

Valid: 100%|██████████| 137/137 [00:05<00:00, 26.59batch/s]


Epoch 83/6000, Train Loss: 0.8229, Train Coeff: -4.2579, Val Loss: 0.8629, Val Coeff: -4.5923, Time: 24.95 sec


Valid: 100%|██████████| 137/137 [00:05<00:00, 26.62batch/s]


Epoch 84/6000, Train Loss: 0.8231, Train Coeff: -4.2419, Val Loss: 0.8677, Val Coeff: -4.6536, Time: 25.01 sec


Train:  96%|█████████▌| 131/137 [00:18<00:00,  6.97batch/s]